# Python II - Assignment 3
This **Home Assignment** is to be submitted and you will be given points for each of the tasks. The first task will recap a lot of the things you learned in this couse (web scraping, regex, pandas, nltk). The second task focuses primarly on graphs.

## Formalities
**Submit in a group of 2-3 people until 06.07.2020 23:59CET. The deadline is strict!**

## Evaluation and Grading
General advice for programming excercises at *CSSH*:
Evaluation of your submission is done semi automatically. Think of it as this notebook being 
executed once. Afterwards, some test functions are appended to this file and executed respectively.

Therefore:
* Submit valid _Python3_ code only!
* We run python 3.8 on our machines, so you can use newer features
* Use external libraries only when specified by task.
* Ensure your definitions (functions, classes, methods, variables) follow the specification if
  given. The concrete signature of e.g. a function usually can be inferred from task description, 
  code skeletons and test cases.
* Ensure the notebook does not rely on current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Double check if your code relies on presence of files or directories other than those mentioned
    in given tasks. Tests run under Linux, hence don't use Windows style paths 
    (`some\path`, `C:\another\path`). Also, use paths only that are relative to and within your
    working directory (OK: `some/path`, `./some/path`; NOT OK: `/home/alice/python`, 
    `../../python`).
* Keep your code idempotent! Running it or parts of it multiple times must not yield different
  results. Minimize usage of global variables.
* Ensure your code / notebook terminates in reasonable time.

**There's a story behind each of these points! Don't expect us to fix your stuff!**

Regarding the scores, you will get no points for a task if:
- your function throws an unexpected error (e.g. takes the wrong number of arguments)
- gets stuck in an infinite loop
- takes much much longer than expected (e.g. >1s to compute the mean of two numbers)
- does not produce the desired output (e.g. returns an descendingly sorted list even though we asked for ascending, returns the mean and the std even though we asked for the mean only, only prints the output instead of returning it!)
- ...

# Task 1: Sentiment of Tweets (6 points)

## b) Load dataset from the internet (2.0)
Consider the twitter data provided in https://github.com/zfz/twitter_corpus/blob/master/full-corpus.csv.

Write a function `load_dataset()` that gets the dataset from the internet and loads it into a dataframe with two columns `'gt_sentiment'` and `'original_tweet'` that stores the sentiment and (unprocessed) tweet content from that dataset. It returns that dataframe. Use the requests library to fetch the csv file from the internet. Thereby drop all tweets with "irrelevant" sentiment.

In [1]:
import requests
from io import StringIO
import pandas as pd
def load_dataset():
    # the url of the raw CSV 
    CSV_URL = "https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv"
    page=requests.get(CSV_URL).content
    df=pd.read_csv(StringIO(page.decode('utf-8')))

    df = df.rename(columns={'Sentiment': 'gt_sentiment', 'TweetText': 'original_tweet'})
    data = df[['gt_sentiment','original_tweet']]
    # remove irrelevant tweets
    data = data[data['gt_sentiment']!='irrelevant']
    return data
    

In [2]:
# import csv
# CSV_URL = "https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv"
# with requests.Session() as s:
#     download = s.get(CSV_URL)

#     decoded_content = download.content.decode('utf-8')

#     cr = csv.reader(decoded_content.splitlines(), delimiter=',')
#     my_list = list(cr)

# pd.DataFrame(my_list[1:],columns=my_list[0])

In [3]:
df = load_dataset()
df.head()
# example dataframe:
# 	gt_sentiment 	original_tweet
#0 	positive 	Now all @Apple has to do is get swype on the i...
#1 	positive 	@Apple will be adding more carrier support to ...
#2 	positive 	Hilarious @youtube video - guy does a duet wit...
#3 	positive 	@RIM you made it too easy for me to switch to ...
#4 	positive 	I just realized that the reason I got into twi...

ConnectionError: HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /zfz/twitter_corpus/master/full-corpus.csv (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000024CF3605D08>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## b) Clean tweets (1.5 +0.5)
Write a function `clean_tweet(tweet)` that takes a tweet (as string) as input and removes usernames, hashtags and links. Additionally provide a function `clean_all_tweets(df)` that takes a dataframe like above and applies the `clean_tweet` function to all of them. It returns the same dataframe with now an additional column `cleaned_tweet`.

In [ ]:
import re

In [ ]:
def clean_tweet(tweet):

    user_handles = re.findall('@\S+',tweet)
    cleaned_tweet = re.sub('@\S+','',tweet)
    
    hashtags = re.findall('#\S+',tweet)
    cleaned_tweet = re.sub('#\S+','',cleaned_tweet)
    
    # links can be started by "http://" or https:// so we have to remove both of them
    links1 = re.findall('http://\S+',tweet)
    cleaned_tweet = re.sub('http://\S+','',cleaned_tweet)

    links2 = re.findall('https://\S+',tweet)
    cleaned_tweet = re.sub('https://\S+','',cleaned_tweet)
    # concatenate link1 & link2
    links = links1 + links2

    
    
    return cleaned_tweet.strip(), user_handles, hashtags, links

In [ ]:
example1="Digital X Worldwide | Today Is Steve Jobs Day In California @apple http://t.co/QSCHuMIN"
example2="Wow. Great deals on refurbed #iPad (first gen) models. RT: Apple offers great deals on refurbished 1st-gen iPads http://t.co/ukWOKBGd @Apple"

In [ ]:
print(clean_tweet(example1))
# ('Digital X Worldwide | Today Is Steve Jobs Day In California', ['@apple'], [], ['http://t.co/QSCHuMIN'])
print(clean_tweet(example2))
# ('Wow. Great deals on refurbed  (first gen) models. RT: Apple offers great deals on refurbished 1st-gen iPads', ['@Apple'], ['#iPad'], ['http://t.co/ukWOKBGd'])

In [ ]:
def clean_all_tweets(df):
    df['cleaned_tweet'] = df['original_tweet'].apply(lambda x :clean_tweet(x)[0])
    return df
clean_all_tweets(df)

## c) Predict the sentiment (1.5)
Write a function `calculate_sentiment(df)` that calculate the sentiment of the tweet using the vader sentiment classifier from nltk. Use the **compound** score of the `polarity_scores`. It is between -1 and 1. You can classify the sentiment as - 

```
[-1,-0.2) -> negative

[-0.2,0.2] -> neutral

(0.2,1.0] -> positive```

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def calculate_sentiment(df):
    sia_polarity = SentimentIntensityAnalyzer().polarity_scores
    df['sentiment'] = df['cleaned_tweet'].apply(lambda x : sia_polarity(x)['compound'])

    def check_sentiment(x):
        if  x < -.2:
            return 'negative'
        elif 0.2 < x:
            return 'positive'
        else:
            return 'neutral'

    df['sentiment'] = df['sentiment'].apply(lambda x : check_sentiment(x))
    
    return df

In [ ]:
calculate_sentiment(df)

## d) Tiny evaluation (0.5)
Write a function `accuracy` that calculates the accuracy of the sentiment classifier by calculating the fraction of cases where the calculated sentiment matches the original sentiment.

In [ ]:
def accuracy(df):
    
    df['match'] = df['gt_sentiment'] == df['sentiment']
    # we could also use mean() here
    df['match'].sum()/len(df['match'])
    
    return df['match'].sum()/len(df['match'])

accuracy(df)
# output
# 0.5791471962616822

In [ ]:
""" This solution contains word-preprocessing which improves the accuracy to "0.6810747663551402"
"""
from nltk import word_tokenize , sent_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
puncts = string.punctuation

# df = load_dataset()
# clean_all_tweets(df)

def accuracy2(df):
    s_words = stopwords.words('english')
    
    def word_preprocess(tweet):
        text_tokens = word_tokenize(tweet.strip())
        # remove the punctuations from the "text_tokens"
        text_tokens_f = [w for w in text_tokens if w not in puncts]
        # remove stopwords
        text_tokens_f = [w for w in text_tokens_f if w.lower() not in s_words]
        # convert words --> sentence
        tweet = ''.join(text_tokens_f)
        
        sia_polarity = SentimentIntensityAnalyzer().polarity_scores
        sentiment = sia_polarity(tweet)['compound']

        def check_sentiment(x):
            if  x < -.2:
                return 'negative'
            elif 0.2 < x:
                return 'positive'
            else:
                return 'neutral'
            
            
        sentiment = check_sentiment(sentiment)
        return sentiment
    
    df['sentiment'] = df['cleaned_tweet'].apply(lambda x : word_preprocess(x))
            
    
    df['match'] = df['gt_sentiment'] == df['sentiment']
    
    # we could also use mean() here
    return df['match'].sum()/len(df['match'])

accuracy2(df)
# output
# 0.5791471962616822
# this solusiton that we got is "0.6810747663551402" accuracy which is 11% more than a naive sentiment with "0.5791471962616822"
# what we understood is in order to get the better sentiment we have to 

# Task 2: Minimum cost spanning-tree (7.0 points)

## a) Read an example file (1)
Consider the data provided in file `'reachability.txt'`. It is consists of a network with nodes representing airports of USA and Canada. Edges are weighted so that there is an edge from city i to city j if the estimated airline travel time is less than a threshold. The travel time includes estimated stopover delays.
Write a function `read_graph(filename)` that reads the file at `filename` an returns a directed networkx graph with weights.

In [4]:
import pandas as pd
import networkx as nx
def read_graph(filename):
    """ we manually counted the number of lines which contain text and skip those
    rows but we could also use the below lines of code to find the lines which start 
    with "#" then skip them:
    
    table = []
    filename = 'reachability.txt'
    with open(filename,'r') as f:
        for line in f:
            table += re.findall('#.+',line)
            
    skiprows = len(table)
    """
    # "column_name" : also we added the names of the columns which is unnecessary 
    column_name = ['FromNodeId','ToNodeId','Weight']
    df = pd.read_table(filename,delimiter=' ',header=None,skiprows=6,names=column_name)

    g = nx.DiGraph()
    for i in df.itertuples():
        g.add_edge(int(i[1]), int(i[2]), weight=-int(i[3]))
        
    
    return g

In [5]:
G_large = read_graph('reachability.txt')
assert G_large.is_directed()
l = list(G_large.edges(data=True))
l[:5]
# Output
# [(27, 0, {'weight': 757}),
#  (27, 1, {'weight': 291}),
#  (27, 2, {'weight': 295}),
#  (27, 3, {'weight': 341}),
#  (27, 4, {'weight': 359})]

[(27, 0, {'weight': 757}),
 (27, 1, {'weight': 291}),
 (27, 2, {'weight': 295}),
 (27, 3, {'weight': 341}),
 (27, 4, {'weight': 359})]

## b) Make the Graph symmetric (1.5)
The graph is directed and the weights are asymmetric. Write a function `make_symmetric(G_in)` that takes such a directed graph with weights as input and converts it to an undirected and symmetric network graph. You should use the following simple strategy:

If for two nodes u,v , edge u->v has a weight $w_1$ and v->u has weight $w_2$ then symmetric network should have one edge u,v with weight $(w_1 + w_2)/2$. Note that in the final graph each edge should be counted only once.

In [13]:
def make_symmetric(G_in):
    pass

# Create example Graph
G1 = nx.DiGraph()
G1.add_edge(0,1,weight=10)
G1.add_edge(1,0,weight=5)
G1.add_edge(0,2,weight=3)
G1.add_edge(3,0,weight=1)
G1.add_edge(2,3,weight=19)
G1.add_edge(3,2,weight=11)
G1.add_edge(3,4,weight=0.1)
G_out = make_symmetric(G1)
print(G_out.edges(data=True))
# [(0, 1, {'weight': 7.5}), (0, 2, {'weight': 3}), (0, 3, {'weight': 1}), (2, 3, {'weight': 15.0}), (3, 4, {'weight': 0.1})]

[(0, 1, {'weight': 7.5}), (0, 2, {'weight': 3}), (0, 3, {'weight': 1}), (2, 3, {'weight': 15.0}), (3, 4, {'weight': 0.1})]


## c) Helper class: ConnectedComponentsTracker (2.5)
It is sometimes useful to keep track of the connected components https://en.wikipedia.org/wiki/Component_(graph_theory) when building a graph. Therefor write a class `ConnectedComponentsTracker` that keeps track of connected components in its attribute set. This attribute is a list of sets, where if two nodes u,v are in the same sets they are connected, otherwise they are not.
It should have these methods:
1. The constructor should initialize the `self.sets` to an empty list
2. `find_set(self, u)` that returns the set that contains u or `len(self.sets)` if u is not in any of the sets
3. `add_edge(self, u, v)` that updates the sets of the connected component to account for the new edge `(u,v)`
4. `forms_loop(self, u, v)` that returns True if inserting the undirected edge `(u,v)` would create a loop or is an existing edge, False otherwise

The ConnectedComponentsTracker is only intended for use with symmetric Graphs so any edge u,v is can be considered symmetric. You can find example output for a valid `ConnectedComponentsTracker` below.

In [15]:
# Test your class here
c=ConnectedComponentsTracker()

c.add_edge(0,1)
print(0, c.sets)
print(0.1, c.forms_loop(2,1))
c.add_edge(1,2)
print(1, c.sets)

c.add_edge(3,2)
print(2, c.sets)
print(2.1, c.forms_loop(1,3))

c.add_edge(1,3)
print(3, c.sets)

c.add_edge(4,5)
print(4, c.sets)

c.add_edge(5,6)
print(5, c.sets)
print(5.1, c.forms_loop(4,6))
print(5.2, c.forms_loop(1,6))
print(5.3, c.forms_loop(1,10))

c.add_edge(2,6)
print(6, c.sets)

c.add_edge(9,10)
print(7, c.sets)

c.add_edge(10,3)
print(8, c.sets)

# --- Output ---
# Note that the order in a set is not deterministic!
# 0   [{0, 1}]
# 0.1 False
# 1   [{0, 1, 2}]
# 2   [{0, 1, 2, 3}]
# 2.1 True
# 3   [{0, 1, 2, 3}]
# 4   [{0, 1, 2, 3}, {4, 5}]
# 5   [{0, 1, 2, 3}, {4, 5, 6}]
# 5.1 True
# 5.2 False
# 5.3 False
# 6   [{0, 1, 2, 3, 4, 5, 6}]
# 7   [{0, 1, 2, 3, 4, 5, 6}, {9, 10}]
# 8   [{0, 1, 2, 3, 4, 5, 6, 9, 10}]


0 [{0, 1}]
0.1 False
1 [{0, 1, 2}]
2 [{0, 1, 2, 3}]
2.1 True
3 [{0, 1, 2, 3}]
4 [{0, 1, 2, 3}, {4, 5}]
5 [{0, 1, 2, 3}, {4, 5, 6}]
5.1 True
5.2 False
5.3 False
6 [{0, 1, 2, 3, 4, 5, 6}]
7 [{0, 1, 2, 3, 4, 5, 6}, {9, 10}]
8 [{0, 1, 2, 3, 4, 5, 6, 9, 10}]


## d) Kruskal's algorithm (1.5)
Obtain minimum cost spanning tree (https://en.wikipedia.org/wiki/Minimum_spanning_tree) using Kruskal's algorithm. The algorithm is given below - 

1. Sort all the edges in non-decreasing order of their weight.

2. Pick the smallest edge (i.e., the edge with the lowest weight). Check if it forms a cycle with the spanning tree formed so far (i.e., all edges selected so far). If cycle is not formed, include this edge. Else, discard it.

3. Repeat step#2 until there are (V-1) edges in the spanning tree. 

V -> number of nodes in the graph

Now write a function `minSpanTree` that takes an undirected networkx Graph with weights and returns a new undirected networkx Graph with only those edges that you obtained from Kruskal's algorithm.

In [16]:
def minSpanTree(G_in):
    pass

G = minSpanTree(G_out)
G.edges(data=True)
# [(3, 4, {'weight': 0.1}), (3, 0, {'weight': 1}), (0, 2, {'weight': 3}), (0, 1, {'weight': 7.5})]

EdgeDataView([(3, 4, {'weight': 0.1}), (3, 0, {'weight': 1}), (0, 2, {'weight': 3}), (0, 1, {'weight': 7.5})])

Use the `minimum_spanning_tree` function of networkx to obtain the minimum cost spanning tree for the graph.

In [17]:
def minSpanTreeNetworkx(G):
    pass
G_netx = minSpanTreeNetworkx(G_out)

## e) Compare the results (0.25+0.25)
Compare your minimal spanning tree with the one from networkx.
Therefor write a function `compare_edges(G, G_netx)` that returns the extra and missing edges for the two graphs.
Additionally write a function `get_total_span(G)` that returns the total span (sum of all weights) for a Graph.
You can use these functions to verify your implementation. Make sure that if there are missing or extra edges that the total span of your implementation is the same as that of networkx.

In [18]:
def get_total_span(G):
    return 0
    

def compare(G, G_netx):
    extra_edges = []
    missing_edges = []
    # If there is an edge in G but it is not present in G_netx... add to extra_edges
    # If there is an edge in G_netx but it is not present in G... add to missing_edges
    
    return extra_edges, missing_edges

In [19]:
G_sym = make_symmetric(G_large)
G = minSpanTree(G_sym)
G_netx = minSpanTreeNetworkx(G_sym)
compare(G, G_netx) # There might be some confused edges

([], [])

In [20]:
# If there are confused edges the total span should be the same still
print(get_total_span(G)) 
print(get_total_span(G_netx))
# output
# 23864.0

23864.0
23864.0
